# Text generator based on RNN

### Import libraries

In [ ]:
import tensorflow as tf
import numpy as np
import random

### Configurations

In [ ]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
#TestText="Too young, too simple, sometimes naive. "
TestText="""
Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate—we can not consecrate—we can not hallow—this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us—that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion—that we here highly resolve that these dead shall not have died in vain—that this nation, under God, shall have a new birth of freedom—and that government of the people, by the people, for the people, shall not perish from the earth.
"""
graphPath=r".\graphs"
num_step=len(TestText)-1
batchSize=1
def Encode(string,name=None):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor = tf.one_hot(tensor, depth=len(vocab)+1, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32, name=name)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
X=Encode(TestText)
Y=X[1:]
X=X[:-1]
print(X.shape,Y.shape)

### Create graph

In [ ]:
class MyRNN():
    def __init__(self,inputVectorDim,stateSize=100,scope="RNN",outputVectorDim=None):
        if outputVectorDim==None:
            outputVectorDim=inputVectorDim
        self.__inputVectorDim=inputVectorDim
        self.__outputVetorDim=outputVectorDim
        self.__stateSize=stateSize
        with tf.variable_scope(scope) as modelScope:
            with tf.variable_scope("structure") as structureScope:
                #One batch at a time
                self.inputs=tf.placeholder(dtype=tf.float32,shape=[None,inputVectorDim])
                self.outputs=tf.placeholder(dtype=tf.float32,shape=[None,outputVectorDim])
                self.__cell=tf.contrib.rnn.GRUCell(num_units=stateSize)
                initState = tf.placeholder_with_default(self.__cell.zero_state(batch_size=1,dtype=tf.float32),
                                                        shape=[None,self.__stateSize])
                #S20:State to output
                self.__wS2O=tf.get_variable(name="Weight_s2o",shape=[stateSize,outputVectorDim],
                                            initializer=tf.truncated_normal_initializer())
                self.__bS2O=tf.get_variable(name="Bias_s2o",shape=[1,outputVectorDim],
                                            initializer=tf.truncated_normal_initializer())
                #add a "batch dimension" to the front
                cellInputs=tf.reshape(tensor=self.inputs,shape=[1,-1,inputVectorDim])
                outputs,finalState=tf.nn.dynamic_rnn(cell=self.__cell,inputs=cellInputs,dtype=tf.float32,initial_state=initState)
                outputs=tf.reshape(tensor=outputs, shape=[-1, stateSize])#Remove the batch dimension ((
                self.netOutputs=tf.add(tf.matmul(outputs,self.__wS2O),self.__bS2O)
            with tf.name_scope("training"):
                loss=tf.losses.softmax_cross_entropy(logits=self.netOutputs,onehot_labels=self.outputs)
                loss=tf.reduce_mean(loss,name="loss")
                self.optimizer=tf.train.AdamOptimizer(learning_rate=0.001,name="optimizer").minimize(loss)
            with tf.name_scope("summary") as sumScope:
                lossSum=tf.summary.scalar(tensor=loss,name="loss")
                self.summary_op=tf.summary.merge(tf.get_collection(tf.GraphKeys.SUMMARIES,sumScope))
    def onlineInference(self,intputTensor,timeSteps=100,state=None,seed=None):
        if state==None :
            seed=self.__cell.zero_state(batch_size=1,dtype=tf.float32)
        if seed==None:
            seed=tf.one_hot([random.randint(0,self.__inputVectorDim)],
                            depth=self.__inputVectorDim,on_value=1.0,off_value=0.0,axis=-1,dtype=tf.float32)
        input_=seed
        with tf.Session() as sess:
            for length in range(timeSteps):
                outputs,state=tf.nn.dynamic_rnn(cell=self.__cell,initial_state=state,dtype=tf.float32,inputs=input_)
                outputs=tf.reshape(tensor=outputs, shape=[-1, stateSize])
                outputs=tf.add(tf.matmul(outputs,self.__wS2O)+self.__bS2O)
                seq.append(outputs)
                inputs_=outputs
        return seq,state

In [ ]:
Test=MyRNN(scope="RNN",inputVectorDim=len(vocab)+1)

In [ ]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    result=sess.run(Test.netOutputs,{Test.inputs:X,Test.outputs:Y})
    writer.close()
print(Decode(result))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        sess.run(Test.optimizer,feed_dict={Test.inputs:X,Test.outputs:Y})
        if i%20==0:
            result=sess.run(Test.netOutputs,{Test.inputs:X})
            print(Decode(result))